In [ ]:
import cv2 as cv
import numpy as np 
import sys
import os 
import time
import tensorflow as tf
import pygame

In [ ]:
# Model Used to detect face and eyes 
faceCascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")
eyeCascade = cv.CascadeClassifier("haarcascade_eye.xml")

# Model used to predict eyes states
eye_model = tf.keras.models.load_model('Training Results/CNN Model-2/75 Epoch/modelCNN_2_75Epoch.h5')

In [ ]:
# FUnction to Predict eyes state
def getPrediction(roi) :
    if roi != []:
        resize = cv.resize(roi, (24,24))
        input = resize.reshape(-1, 24, 24, 1)
        input = input/255.0
        pred = eye_model.predict(input)
        prediction = np.round(pred)
    
    return prediction

In [ ]:
# function to play alarm after eyes closed 3 seconds in a row
def playAlarm () :
    pygame.mixer.init()
    sound_file = "sound/alarm.wav"
    pygame.mixer.music.load(sound_file)
    pygame.mixer.music.set_volume(0.2)
    pygame.mixer.music.play(-1)

In [ ]:
# function to calculate closed eyes time
def counterTime(start_time):
    elapsed_time = time.time() - start_time
    seconds = int(elapsed_time)
    return elapsed_time

## Main Program to Detect Drowsiness in Real-Time

In [ ]:
# Configuring webcam 
capture = cv.VideoCapture(1)
capture.set(3,640) #set width
capture.set(4,480) #set height

# inisiate variable to counter time and alarm state
counter = 0
play = True

# inisiating varible to average calculate fps
start_fps = time.time()
frame_count = 0
fps_list = []

# inisiate variable to calculate system inference speed values 
t_face_list = []
t_lEye_list = []
t_rEye_list = []
t_p_lEye_list = []
t_p_rEye_list = []

# inisiate the state of eyes detection
r_detect = False
l_detect = False

# exit the system if video can't be opened 
if not capture.isOpened():
  print("Could not open video")
  sys.exit()

#capturing the video
while(True):
    
    # break if video not captured
    ret,frame = capture.read()
    if not ret : 
        break

    # Showing the FPS
    frame_count += 1
    elapsed_fps = time.time() - start_fps
    fps = frame_count / elapsed_fps
    fps_list.append(fps)
    cv.putText(frame, f'FPS: {fps:.2f}', (10, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Converting Frame to Grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        
    # initiate the time to calculate face detection inference time
    t0_face = time.time()
    
    # Detecting the face
    print("detect face")
    face = faceCascade.detectMultiScale( gray, minNeighbors = 15, scaleFactor = 1.1, minSize = (50,50) ) 
    
    # write the bounding box around the detected face
    for (x,y,w,h) in face:
        cv.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    
    # Calculate the inference time of left eye detection time   
    t1_face = time.time()   
    t_face = t1_face - t0_face 
    t_face_list.append(t_face)

    
    if face != () :    
        
        # Create the Eyes Detection RoI      
        rRoi_gray = gray[y:y+(h*3//4), x:x+(w//2)] 
        lRoi_gray = gray[y:y+(h*3//4), x+(w//2):x+w] 
        
        # Create the RoI to write the bounding box for detected eyes
        r_roi = frame[y:y+(h*3//4), x:x+(w//2)] 
        l_roi = frame[y:y+(h*3//4), x+(w//2):x+w] 
              
        # initiate the time to calculate left eye detection inference time
        t0_lEye = time.time()
        
        # Detecting the left eye  
        print("detect left Eye")
        l_eye = eyeCascade.detectMultiScale(lRoi_gray, minNeighbors = 10) 
        
        # write the bounding box around the detected left eye 
        for (x1,y1,w1,h1) in l_eye:
            cv.rectangle(l_roi,(x1,y1),(x1+w1,y1+h1),(0,255,0),2) 
            l_detect = True
            
        # Calculate the inference time of left eye detection time
        t1_lEye = time.time()   
        t_lEye = t1_lEye - t0_lEye 
        t_lEye_list.append(t_lEye)
        
        # initiate the time to calculate right eye detection inference time
        t0_rEye = time.time()
        
        # Detecting the right eye
        print("detect right eye")
        r_eye = eyeCascade.detectMultiScale(rRoi_gray, minNeighbors = 10)  
        
        # write the bounding box around the detected left eye
        for (x2,y2,w2,h2) in r_eye:
            cv.rectangle(r_roi,(x2,y2),(x2+w2,y2+h2),(0,0,255),2)           
            r_detect = True
        
        # Calculate the inference time of right eye detection time    
        t1_rEye = time.time()   
        t_rEye = t1_rEye - t0_rEye
        t_rEye_list.append(t_rEye)
        
        # Predict eyes state if both eyes are detected
        if (r_detect == True) & (l_detect == True):
            
            # create the eyes RoI to be predicted 
            lEye_roi = lRoi_gray[y1:y1+h1, x1:x1+w1]
            rEye_roi = rRoi_gray[y2:y2+h2, x2:x2+w2]
            
            # initiate the time to calculate left eye prediction inference time 
            t0_p_lEye = time.time()
            
            # predict the left eye state 
            print("predict left eye")
            l_pred = getPrediction(lEye_roi)
            
            # Calculate the inference time of left eye prediction time 
            t1_p_lEye = time.time()   
            t_p_lEye = t1_p_lEye - t0_p_lEye 
            t_p_lEye_list.append(t_p_lEye)
            
            # initiate the time to calculate right eye prediction inference time 
            t0_p_rEye = time.time()
            
            # predict the right eye state 
            print("predict right eye")
            r_pred = getPrediction(rEye_roi)
            
            # Calculate the inference time of right eye prediction time 
            t1_p_rEye = time.time()   
            t_p_rEye = t1_p_rEye - t0_p_rEye 
            t_p_rEye_list.append(t_p_rEye)

            # if both eyes are open 
            if (l_pred == 1) & (r_pred == 1) :
                
                # reset the counter and stop the alarm if eyes open
                counter = 0
                play = True  
                pygame.mixer.quit()
                
                # Show a 'Closed Eyes' text
                cv.putText(frame, "Open Eyes", (460,40), cv.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv.LINE_4)                
            
            # if both eyes are open  
            elif (l_pred == 0) & (r_pred == 0) :
                # Calculate the closed eyes times
                if counter == 0 :
                    start_count = time.time()
                    counter = 1
                elapsed_count = time.time() - start_count
                seconds = int(elapsed_count)
                
                # Show a 'Closed Eyes' text, the times eyes closed, and Rectangle arround the frame                
                cv.putText(frame, "Closed Eyes", (440,40), cv.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv.LINE_4)
                cv.putText(frame, str(seconds), (10,460), cv.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2, cv.LINE_4)
                cv.rectangle(frame, (0, 0), (640, 480), (0, 0, 255), 7)
                
                # Activating The Alarm if eyes closed 3 seconds in a row  
                if (seconds > 3) & (play == True) :
                    play = False
                    playAlarm()
    
    # Show the video captured                  
    cv.imshow('frame',frame)

    # Break the video if 'q' is pressed
    if cv.waitKey(1) & 0xFF == ord('q'):
        pygame.mixer.quit()
        break

capture.release()
cv.destroyAllWindows()

Showing System inference speed

In [ ]:
#calculate the average FPS
average_fps = sum(fps_list) / len(fps_list)
print("Average FPS: {:.2f}".format(average_fps))

#calculate the average face detection
average_face = sum(t_face_list) / len(t_face_list)
print("Average Face detection:", np.round(average_face, decimals= 3) )

#calculate the average left eye detection
average_lEye = sum(t_lEye_list) / len(t_lEye_list)
print("Average Left Eye Detection: ", np.round(average_lEye, decimals=3) )

#calculate the average right eye detection
average_rEye = sum(t_rEye_list) / len(t_rEye_list)
print("Average Right Eye Detection: ", np.round(average_rEye, decimals=3) )

#calculate the average left eye prediction
average_p_lEye = sum(t_p_lEye_list) / len(t_p_lEye_list)
print("Average Left Eye Prediction: ", np.round(average_p_lEye, decimals=3) )

#calculate the average right eye prediction
average_p_rEye = sum(t_p_rEye_list) / len(t_p_rEye_list)
print("Average Right Eye Prediction: ", np.round(average_p_rEye, decimals=3) )
